In [1]:
import warnings
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.svm import SVC

warnings.filterwarnings("ignore")

## FGNet-LOPO -- Hierarchical Model

In [2]:
df_fgnet = pd.read_csv("data/FGNet-LOPO.csv")
df_fgnet["ageclass"] = df_fgnet.age.apply(
    lambda r: 0 if r < 18 else 1
).astype(int)

df_fgnet['ageclass_ext'] = pd.cut(
    df_fgnet.age,
    bins=[0,13,16,20,24,28,32,36,40,100],
    labels=[0,1,2,3,4,5,6,7,8],
    include_lowest=True,
    ordered=False,
)

print(df_fgnet.columns)
print(f' <18: {df_fgnet[df_fgnet.age < 18].shape[0]}\n>=18: {df_fgnet[df_fgnet.age >= 18].shape[0]}')
print(df_fgnet[["ageclass"]].value_counts())
print(df_fgnet[["ageclass_ext"]].value_counts())
df_fgnet[["age", "ageclass", "ageclass_ext"]].sample(20)

Index(['b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'b8', 'b9', 'b10',
       ...
       'b105', 'b106', 'b107', 'b108', 'b109', 'age', 'ID', 'Gender_0M_1F',
       'ageclass', 'ageclass_ext'],
      dtype='object', length=114)
 <18: 640
>=18: 362
ageclass
0           640
1           362
dtype: int64
ageclass_ext
0               513
2               118
1                99
3                64
8                60
4                51
5                38
6                36
7                23
dtype: int64


,age,ageclass,ageclass_ext
680,0,0,0
745,53,1,8
420,5,0,0
767,4,0,0
962,11,0,0
504,19,1,2
582,0,0,0
414,35,1,6
339,34,1,6
659,13,0,0


In [3]:
def df_shape_table(*df_dicts):
    title_str = f"{'DataFrame':>15} | {'Shape':15}"
    print(title_str)
    for df_dict in df_dicts:
        print(f"{''.join(['-'] * 25):^33}")
        for name, df in df_dict.items():
            print(f"{name:>15} | { str(df.shape) :<15}")

In [4]:
df_yng = df_fgnet.iloc[df_fgnet[df_fgnet.ageclass == 0].index, :]
df_old = df_fgnet.iloc[df_fgnet[df_fgnet.ageclass == 1].index, :]

X = df_fgnet.drop(["age", "ID", "Gender_0M_1F", "ageclass", "ageclass_ext"], axis=1)
X_yng = df_yng.drop(["age", "ID", "Gender_0M_1F", "ageclass", "ageclass_ext"], axis=1)
X_old = df_old.drop(["age", "ID", "Gender_0M_1F", "ageclass", "ageclass_ext"], axis=1)

y = df_fgnet[["ID", "age", "ageclass", "ageclass_ext", "Gender_0M_1F"]]
y_yng = df_yng[["ID", "age", "ageclass", "ageclass_ext", "Gender_0M_1F"]]
y_old = df_old[["ID", "age", "ageclass", "ageclass_ext", "Gender_0M_1F"]]

df_shape_table({'X': X, 'X Young': X_yng, 'X Old': X_old},
               {'y': y, 'y Young': y_yng, 'y Old': y_old})

      DataFrame | Shape          
    -------------------------    
              X | (1002, 109)    
        X Young | (640, 109)     
          X Old | (362, 109)     
    -------------------------    
              y | (1002, 5)      
        y Young | (640, 5)       
          y Old | (362, 5)       


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)
X_yng_train, X_yng_test, y_yng_train, y_yng_test = train_test_split(X_yng, y_yng, test_size=.2)
X_old_train, X_old_test, y_old_train, y_old_test = train_test_split(X_old, y_old, test_size=.2)

df_shape_table(
    {'X_train': X_train, 'X_test': X_test, 'y_train': y_train, 'y_test': y_test},
    {'X_yng_train': X_yng_train, 'X_yng_test': X_yng_test, 'y_yng_train': y_yng_train, 'y_yng_test': y_yng_test},
    {'X_old_train': X_old_train, 'X_old_test': X_old_test, 'y_old_train': y_old_train, 'y_old_test': y_old_test}
)

      DataFrame | Shape          
    -------------------------    
        X_train | (801, 109)     
         X_test | (201, 109)     
        y_train | (801, 5)       
         y_test | (201, 5)       
    -------------------------    
    X_yng_train | (512, 109)     
     X_yng_test | (128, 109)     
    y_yng_train | (512, 5)       
     y_yng_test | (128, 5)       
    -------------------------    
    X_old_train | (289, 109)     
     X_old_test | (73, 109)      
    y_old_train | (289, 5)       
     y_old_test | (73, 5)        


In [6]:
model_metrics = {}

for i in range(10):
    model_metrics[i] = {
        "clf_true": [],
        "clf_pred": [],
        "reg_true": [],
        "reg_pred": [],
        "total": 0,
        "score": 0,
        "error": 0.
    }
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)
    X_yng_train, X_yng_test, y_yng_train, y_yng_test = train_test_split(X_yng, y_yng, test_size=.2)
    X_old_train, X_old_test, y_old_train, y_old_test = train_test_split(X_old, y_old, test_size=.2)
    
    svm = SVC(C=1000, gamma=.01, kernel='rbf').fit(X_train, y_train["ageclass"])
    reg_yng = Ridge(alpha=.1).fit(X_yng_train, y_yng_train["age"])
    reg_old = Ridge(alpha=.05).fit(X_old_train, y_old_train["age"])
    
    for idx in X_test.index:
        row = X_test.loc[idx].values.reshape(1, -1)
        
        age_true = y_test.loc[idx, "age"]
        ageclass_true = y_test.loc[idx, "ageclass"]
        ageclass_pred = svm.predict(row)
        
        if ageclass_pred not in [0, 1]:
            raise ValueError("ageclass must be either 0 or 1")
            
        age_pred = reg_yng.predict(row) if ageclass_pred == 0 else reg_old.predict(row)
        
        model_metrics[i]["error"] += abs(age_pred - age_true)
        model_metrics[i]["score"] += 1 if np.round(age_pred) == age_true else 0
        model_metrics[i]["total"] += 1
        model_metrics[i]["clf_true"].append(ageclass_true)
        model_metrics[i]["clf_pred"].append(ageclass_pred)
        model_metrics[i]["reg_true"].append(age_true)
        model_metrics[i]["reg_pred"].append(age_pred)
        
    print(model_metrics[i]["error"] / model_metrics[i]["total"])

[3.51509702]
[3.7652085]
[3.3261141]
[3.82194678]
[3.63535264]
[3.60933187]
[4.19393772]
[3.52177319]
[4.0382271]
[3.38918981]


### Classification/Regression Metrics

In [35]:
from sklearn import metrics as m

def calculate_metrics(model_metrics, multi_class=False):
    clf_metrics = {
        'Accuracy': [],
        'Precision': [],
        'Recall': [],
        'F1 Score': [],
        'Jaccard': [],
    }

    reg_metrics = {
        "MAE": [],
        "MSE": [],
        "RMSE": [],
        "R2": [],
        "MAPE": [],
        "Median AE": [],
        "Max Error": [],
    }

    index = []
    for idx, metrics in model_metrics.items():
        clf_true, clf_pred = metrics["clf_true"], metrics["clf_pred"]
        reg_true, reg_pred = metrics["reg_true"], metrics["reg_pred"]
        
        if not multi_class: 
            clf_metrics["Accuracy"].append(m.accuracy_score(clf_true, clf_pred))
            clf_metrics["Precision"].append(m.precision_score(clf_true, clf_pred))
            clf_metrics["Recall"].append(m.recall_score(clf_true, clf_pred))
            clf_metrics["F1 Score"].append(m.f1_score(clf_true, clf_pred))
            clf_metrics["Jaccard"].append(m.jaccard_score(clf_true, clf_pred))
            
        else:
            clf_metrics["Accuracy"].append(m.accuracy_score(clf_true, clf_pred))
            clf_metrics["Precision"].append(m.precision_score(clf_true, clf_pred, average=None))
            clf_metrics["Recall"].append(m.recall_score(clf_true, clf_pred, average=None))
            clf_metrics["F1 Score"].append(m.f1_score(clf_true, clf_pred, average=None))
            clf_metrics["Jaccard"].append(m.jaccard_score(clf_true, clf_pred, average=None))
        
        reg_metrics["MAE"].append(m.mean_absolute_error(reg_true, reg_pred))
        reg_metrics["MSE"].append(m.mean_squared_error(reg_true, reg_pred))
        reg_metrics["RMSE"].append(m.mean_squared_error(reg_true, reg_pred, squared=False))
        reg_metrics["R2"].append(m.r2_score(reg_true, reg_pred))
        reg_metrics["MAPE"].append(m.mean_absolute_percentage_error(reg_true, reg_pred))
        reg_metrics["Median AE"].append(m.median_absolute_error(reg_true, reg_pred))
        reg_metrics["Max Error"].append(m.max_error(reg_true, reg_pred))
            
        index.append(idx)
            
    clf_metrics = pd.DataFrame(clf_metrics, index=pd.Index(index)).round(4)
    reg_metrics = pd.DataFrame(reg_metrics, index=pd.Index(index)).round(4)
    
    return pd.concat({'Regression': reg_metrics, 'Classification': clf_metrics}, axis=1)


calculate_metrics(model_metrics)

Regression                                                               \
          MAE       MSE     RMSE      R2          MAPE Median AE Max Error   
1     12.0791  232.4881  15.2476 -0.3590  9.077000e-01    9.1628   28.8372   
2      9.8139  139.6483  11.8173 -0.2013  8.392000e-01    9.0000   23.8372   
3     26.2538  926.0077  30.4304 -2.9114  5.804000e-01   28.3372   46.8372   
4     25.6705  877.1143  29.6161 -3.0209  5.856000e-01   24.3372   48.8372   
5     25.2008  789.3127  28.0947 -4.1177  5.938000e-01   25.8372   46.8372   
..        ...       ...      ...     ...           ...       ...       ...   
78     6.7675   65.6431   8.1020 -2.0891  3.986479e+15    6.6628   14.1628   
79     7.6628   74.9688   8.6585 -3.6135  4.555975e+15    7.6628   14.1628   
80     7.4485   74.2561   8.6172 -2.9549  4.555975e+15    7.6628   14.1628   
81     8.4128   85.1297   9.2266 -4.9307  5.315305e+15    8.6628   14.1628   
82     9.1691  102.6690  10.1326 -0.2546  5.798514e+15    8.8372   16.8372   

   Classification                                     
         Accuracy Precision  Recall F1 Score Jaccard  
1          0.8000     0.750  1.0000   0.8571  0.7500  
2          0.7500     1.000  0.5556   0.7143  0.5556  
3          1.0000     1.000  1.0000   1.0000  1.0000  
4          1.0000     1.000  1.0000   1.0000  1.0000  
5          1.0000     1.000  1.0000   1.0000  1.0000  
..            ...       ...     ...      ...     ...  
78         1.0000     0.000  0.0000   0.0000  0.0000  
79         1.0000     0.000  0.0000   0.0000  0.0000  
80         1.0000     0.000  0.0000   0.0000  0.0000  
81         1.0000     0.000  0.0000   0.0000  0.0000  
82         0.8182     0.875  0.8750   0.8750  0.7778  

[82 rows x 12 columns]

## FGNet-LOPO -- Hierarchical Model w/ LOPOCV

In [8]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVR

In [9]:
model_metrics = {}

for i in range(10):
    model_metrics[i] = {
        "clf_true": [],
        "clf_pred": [],
        "reg_true": [],
        "reg_pred": [],
        "total": 0,
        "score": 0,
        "error": 0.
    }
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)
    X_yng_train, _, y_yng_train, _ = train_test_split(X_yng, y_yng, test_size=.2)
    X_old_train, _, y_old_train, _ = train_test_split(X_old, y_old, test_size=.2)
    
    svm = SVC(C=1000, gamma = .01, kernel='rbf').fit(X_train, y_train["ageclass"])
    reg_yng = Ridge(alpha=.1).fit(X_yng_train, y_yng_train["age"])
    reg_old = make_pipeline(StandardScaler(), 
                            LinearSVR(epsilon=2.97, tol=1e-6, C=14.1)
                            ).fit(X_old_train, y_old_train["age"])
    
    for idx in X_test.index:
        row = X_test.loc[idx].values.reshape(1, -1)
        
        age_true = y_test.loc[idx, "age"]
        ageclass_true = y_test.loc[idx, "ageclass"]
        ageclass_pred = svm.predict(row)
        
        if ageclass_pred not in [0, 1]:
            raise ValueError("ageclass must be either 0 or 1")
            
        age_pred = reg_yng.predict(row) if ageclass_pred == 0 else reg_old.predict(row)
        
        model_metrics[i]["error"] += abs(age_pred - age_true)
        model_metrics[i]["score"] += 1 if np.round(age_pred) == age_true else 0
        model_metrics[i]["total"] += 1
        model_metrics[i]["clf_true"].append(ageclass_true)
        model_metrics[i]["clf_pred"].append(ageclass_pred)
        model_metrics[i]["reg_true"].append(age_true)
        model_metrics[i]["reg_pred"].append(age_pred)
        
    print(model_metrics[i]["error"] / model_metrics[i]["total"])
    
calculate_metrics(model_metrics)

[3.51297215]
[3.81619426]
[3.72662102]
[4.26724645]
[4.21598398]
[4.09612228]
[3.87345192]
[3.67876119]
[3.65768092]
[4.4468564]


Regression                                                             \
         MAE      MSE    RMSE      R2          MAPE Median AE Max Error   
0     3.5130  31.2704  5.5920  0.7869  4.504452e+14    2.2574   24.6283   
1     3.8162  33.6718  5.8027  0.8184  3.352654e+14    2.5551   25.3229   
2     3.7266  36.1869  6.0156  0.7934  4.468961e+14    2.3786   27.6226   
3     4.2672  62.3898  7.8987  0.6406  3.990710e+14    2.1636   55.6756   
4     4.2160  56.9099  7.5439  0.6319  2.171587e+14    1.9530   38.1252   
5     4.0961  38.8620  6.2339  0.7278  3.136387e+14    2.8259   25.6925   
6     3.8735  34.2165  5.8495  0.8081  3.240398e+14    2.5361   25.9587   
7     3.6788  35.6619  5.9718  0.8070  5.269691e+14    2.0208   32.2825   
8     3.6577  34.0139  5.8321  0.8134  2.469323e+14    2.4316   27.2446   
9     4.4469  57.7575  7.5998  0.6613  1.341925e+15    2.3248   46.7636   

  Classification                                     
        Accuracy Precision  Recall F1 Score Jaccard  
0         0.8856    0.8485  0.8116   0.8296  0.7089  
1         0.8657    0.8472  0.7922   0.8188  0.6932  
2         0.8856    0.8571  0.7941   0.8244  0.7013  
3         0.8259    0.7385  0.7273   0.7328  0.5783  
4         0.8159    0.7500  0.7397   0.7448  0.5934  
5         0.8507    0.8481  0.7882   0.8171  0.6907  
6         0.8806    0.8169  0.8406   0.8286  0.7073  
7         0.8856    0.8356  0.8472   0.8414  0.7262  
8         0.8756    0.8286  0.8169   0.8227  0.6988  
9         0.8557    0.8267  0.7949   0.8105  0.6813

## Multilevel One-Hot

In [10]:
y['ageclass_ext'].unique()

[0, 1, 2, 3, 4, 5, 6, 7, 8]
Categories (9, int64): [0, 1, 2, 3, ..., 5, 6, 7, 8]

In [11]:
model_metrics = {}

for i in range(10):
    model_metrics[i] = {
        "clf_true": [],
        "clf_pred": [],
        "reg_true": [],
        "reg_pred": [],
        "total": 0,
        "score": 0,
        "error": 0.
    }
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)
    
    svm = SVC(C=1000, gamma=.01, kernel='rbf').fit(X_train, y_train['ageclass_ext'])
    reg = [Ridge(alpha=.1),
           make_pipeline(StandardScaler(),
                         LinearSVR(epsilon=2.97, tol=1e-6, C=14.1)),
           Ridge(alpha=.1), Ridge(alpha=.1), Ridge(alpha=.1),
           Ridge(alpha=.1), Ridge(alpha=.1), Ridge(alpha=.1),
           Ridge(alpha=.1)]
    
    for r in range(len(reg)):
        indices = y_train[y_train['ageclass_ext'] == r].index
        reg[r].fit(X_train.loc[indices], y_train.loc[indices,'age'])
        
    # reg_yng = Ridge(alpha=.1).fit(X_yng_train, y_yng_train["age"])
    # reg_old = make_pipeline(StandardScaler(), 
    #                         LinearSVR(epsilon=2.97, tol=1e-6, C=14.1)
    #                         ).fit(X_old_train, y_old_train["age"])
    
    for idx in X_test.index:
        row = X_test.loc[idx].values.reshape(1, -1)
        ageclass_true = y_test.loc[idx, 'ageclass_ext']
        ageclass_pred = svm.predict(row)
        
        # if ageclass_pred not in list(y['ageclass_ext'].unique()):
        #     raise ValueError("ageclass must be in {}".format(str(y['ageclass_ext'].unique())))
        
        age_pred = reg[ageclass_pred[0]].predict(row)
        # age_pred = reg_yng.predict(row) if ageclass_pred == 0 else reg_old.predict(row)
        
        model_metrics[i]["error"] += abs(age_pred - age_true)
        model_metrics[i]["score"] += 1 if np.round(age_pred) == age_true else 0
        model_metrics[i]["total"] += 1
        model_metrics[i]["clf_true"].append(ageclass_true)
        model_metrics[i]["clf_pred"].append(ageclass_pred)
        model_metrics[i]["reg_true"].append(age_true)
        model_metrics[i]["reg_pred"].append(age_pred)
        
    print(model_metrics[i]["error"] / model_metrics[i]["total"])
    
calculate_metrics(model_metrics, multi_class=True)

[8.32911495]
[8.71687075]
[9.34088484]
[9.45545808]
[10.76024747]
[9.64506018]
[9.19085272]
[9.464779]
[9.44008566]
[8.43612668]


Regression                                                      \
         MAE       MSE     RMSE   R2    MAPE Median AE Max Error   
0     8.3291  114.3122  10.6917  0.0  0.6407    6.5324   37.7891   
1     8.7169  134.4617  11.5958  0.0  0.6705    6.6126   42.9068   
2     9.3409  165.4847  12.8641  0.0  0.7185    6.3119   41.8537   
3     9.4555  169.2750  13.0106  0.0  0.7273    6.1523   43.5048   
4    10.7602  216.9447  14.7290  0.0  0.8277    7.3074   45.7273   
5     9.6451  177.5438  13.3246  0.0  0.7419    6.3560   45.0335   
6     9.1909  156.0017  12.4901  0.0  0.7070    6.4171   40.4589   
7     9.4648  158.3555  12.5839  0.0  0.7281    6.8652   38.9414   
8     9.4401  160.2088  12.6574  0.0  0.7262    7.0749   39.8416   
9     8.4361  121.2630  11.0119  0.0  0.6489    6.3814   37.3569   

  Classification                                                     \
        Accuracy                                          Precision   
0         0.5124  [0.8173076923076923, 0.34782608695652173, 0.24...   
1         0.6468  [0.9469026548672567, 0.25, 0.2916666666666667,...   
2         0.5771  [0.8932038834951457, 0.2, 0.22580645161290322,...   
3         0.5920  [0.8653846153846154, 0.2777777777777778, 0.344...   
4         0.5771  [0.897196261682243, 0.23529411764705882, 0.285...   
5         0.5473  [0.8252427184466019, 0.24, 0.4166666666666667,...   
6         0.5821  [0.8942307692307693, 0.30434782608695654, 0.33...   
7         0.5771  [0.8773584905660378, 0.3157894736842105, 0.318...   
8         0.6020  [0.8679245283018868, 0.3333333333333333, 0.411...   
9         0.5970  [0.8440366972477065, 0.375, 0.3125, 0.33333333...   

                                                      \
                                              Recall   
0  [0.85, 0.32, 0.2727272727272727, 0.0, 0.090909...   
1  [0.9385964912280702, 0.35294117647058826, 0.25...   
2  [0.9387755102040817, 0.23529411764705882, 0.26...   
3  [0.8823529411764706, 0.2631578947368421, 0.5, ...   
4  [0.897196261682243, 0.21052631578947367, 0.4, ...   
5  [0.9239130434782609, 0.3333333333333333, 0.322...   
6  [0.9029126213592233, 0.3684210526315789, 0.333...   
7  [0.9029126213592233, 0.3333333333333333, 0.259...   
8  [0.9387755102040817, 0.375, 0.2592592592592592...   
9  [0.92, 0.3157894736842105, 0.4166666666666667,...   

                                                      \
                                            F1 Score   
0  [0.8333333333333334, 0.3333333333333333, 0.255...   
1  [0.9427312775330398, 0.2926829268292683, 0.269...   
2  [0.9154228855721394, 0.2162162162162162, 0.245...   
3  [0.8737864077669903, 0.27027027027027023, 0.40...   
4  [0.897196261682243, 0.2222222222222222, 0.3333...   
5  [0.8717948717948718, 0.27906976744186046, 0.36...   
6  [0.8985507246376813, 0.3333333333333333, 0.333...   
7  [0.8899521531100479, 0.3243243243243243, 0.285...   
8  [0.9019607843137256, 0.35294117647058826, 0.31...   
9  [0.8803827751196172, 0.34285714285714286, 0.35...   

                                                      
                                             Jaccard  
0  [0.7142857142857143, 0.2, 0.14634146341463414,...  
1  [0.8916666666666667, 0.17142857142857143, 0.15...  
2  [0.8440366972477065, 0.12121212121212122, 0.14...  
3  [0.7758620689655172, 0.15625, 0.25641025641025...  
4  [0.8135593220338984, 0.125, 0.2, 0.09090909090...  
5  [0.7727272727272727, 0.16216216216216217, 0.22...  
6  [0.8157894736842105, 0.2, 0.2, 0.0869565217391...  
7  [0.8017241379310345, 0.1935483870967742, 0.166...  
8  [0.8214285714285714, 0.21428571428571427, 0.18...  
9  [0.7863247863247863, 0.20689655172413793, 0.21...

## LOPOCV

In [12]:
import importlib
import pyfgnet.crossval as cv
importlib.reload(cv)

<module 'pyfgnet.crossval' from '/Users/Michael/Documents/work/2021-UNCW-SVSM/face-recognition/pyfgnet/crossval.py'>

In [31]:
model_metrics = {}

# for i in range(10):
#     model_metrics[i] = {
#         "clf_true": [],
#         "clf_pred": [],
#         "reg_true": [],
#         "reg_pred": [],
#         "total": 0,
#         "score": 0,
#         "error": 0.
#     }
    
    # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)
    # X_yng_train, X_yng_test, y_yng_train, y_yng_test = train_test_split(X_yng, y_yng, test_size=.2)
    # X_old_train, X_old_test, y_old_train, y_old_test = train_test_split(X_old, y_old, test_size=.2)

for uid in y["ID"].unique():
    model_metrics[uid] = {
        "clf_true": [],
        "clf_pred": [],
        "reg_true": [],
        "reg_pred": [],
        "total": 0,
        "score": 0,
        "error": 0.
    }

for _ in range(10):
    for train, valid in cv.LOPOCV(shuffle=True).split(X, y):
        X_train, X_valid = X.loc[train], X.loc[valid]
        y_train, y_valid = y.loc[train], y.loc[valid]
        yng, old = y_train[y_train["ageclass"] == 0].index, y_train[y_train["ageclass"] == 1].index
        
        svm = SVC(C=1000, gamma=.01, kernel='rbf').fit(X_train, y_train["ageclass"])
        reg_yng = Ridge(alpha=.1).fit(X_train.loc[yng], y_train.loc[yng, "age"])
        reg_old = Ridge(alpha=.05).fit(X_train.loc[old], y_train.loc[old, "age"])
        
        for idx in X_valid.index:
            row = X_valid.loc[idx].values.reshape(1, -1)
            
            age_true = y_valid.loc[idx, "age"]
            ageclass_true = y_valid.loc[idx, "ageclass"]
            ageclass_pred = svm.predict(row)
            
            if ageclass_pred not in [0, 1]:
                raise ValueError("ageclass must be either 0 or 1")
            
            uid = y_valid.loc[idx, "ID"]
            model_metrics[uid]["error"] += abs(age_pred - age_true)
            model_metrics[uid]["score"] += 1 if np.round(age_pred) == age_true else 0
            model_metrics[uid]["total"] += 1
            model_metrics[uid]["clf_true"].append(ageclass_true)
            model_metrics[uid]["clf_pred"].append(ageclass_pred)
            model_metrics[uid]["reg_true"].append(age_true)
            model_metrics[uid]["reg_pred"].append(age_pred)

In [32]:
lopo_metrics = calculate_metrics(model_metrics)
lopo_metrics

Regression                                                               \
          MAE       MSE     RMSE      R2          MAPE Median AE Max Error   
0     12.0791  232.4881  15.2476 -0.3590  9.077000e-01    9.1628   28.8372   
1      9.8139  139.6483  11.8173 -0.2013  8.392000e-01    9.0000   23.8372   
2     26.2538  926.0077  30.4304 -2.9114  5.804000e-01   28.3372   46.8372   
3     25.6705  877.1143  29.6161 -3.0209  5.856000e-01   24.3372   48.8372   
4     25.2008  789.3127  28.0947 -4.1177  5.938000e-01   25.8372   46.8372   
..        ...       ...      ...     ...           ...       ...       ...   
77     6.7675   65.6431   8.1020 -2.0891  3.986479e+15    6.6628   14.1628   
78     7.6628   74.9688   8.6585 -3.6135  4.555975e+15    7.6628   14.1628   
79     7.4485   74.2561   8.6172 -2.9549  4.555975e+15    7.6628   14.1628   
80     8.4128   85.1297   9.2266 -4.9307  5.315305e+15    8.6628   14.1628   
81     9.1691  102.6690  10.1326 -0.2546  5.798514e+15    8.8372   16.8372   

   Classification                                     
         Accuracy Precision  Recall F1 Score Jaccard  
0          0.8000     0.750  1.0000   0.8571  0.7500  
1          0.7500     1.000  0.5556   0.7143  0.5556  
2          1.0000     1.000  1.0000   1.0000  1.0000  
3          1.0000     1.000  1.0000   1.0000  1.0000  
4          1.0000     1.000  1.0000   1.0000  1.0000  
..            ...       ...     ...      ...     ...  
77         1.0000     0.000  0.0000   0.0000  0.0000  
78         1.0000     0.000  0.0000   0.0000  0.0000  
79         1.0000     0.000  0.0000   0.0000  0.0000  
80         1.0000     0.000  0.0000   0.0000  0.0000  
81         0.8182     0.875  0.8750   0.8750  0.7778  

[82 rows x 12 columns]

In [33]:
lopo_metrics.describe()

Regression                                                              \
             MAE          MSE       RMSE         R2          MAPE  Median AE   
count  82.000000    82.000000  82.000000  82.000000  8.200000e+01  82.000000   
mean    9.752616   167.742798  11.613454  -1.131476  2.751729e+15   8.993195   
std     4.962875   208.288421   5.768567   1.475304  2.711240e+15   4.923169   
min     4.923900    36.804600   6.066700  -6.432300  4.294000e-01   3.837200   
25%     6.767500    64.948925   8.059025  -1.424625  8.853500e-01   6.662800   
50%     7.999400    83.877850   9.158450  -0.473600  3.986479e+15   8.162800   
75%    10.789775   177.693525  13.329875  -0.189450  5.315305e+15   9.122100   
max    31.670500  1210.160500  34.787400  -0.000000  7.972957e+15  29.837200   

                 Classification                                              
       Max Error       Accuracy  Precision     Recall   F1 Score    Jaccard  
count  82.000000      82.000000  82.000000  82.000000  82.000000  82.000000  
mean   20.751844       0.856121   0.604693   0.617365   0.581354   0.494141  
std    10.121042       0.117115   0.396155   0.398151   0.363510   0.338778  
min    10.162800       0.545500   0.000000   0.000000   0.000000   0.000000  
25%    14.162800       0.769200   0.250000   0.270825   0.400000   0.250000  
50%    14.162800       0.851650   0.750000   0.775000   0.714300   0.555600  
75%    26.837200       0.935275   1.000000   1.000000   0.857100   0.750000  
max    54.837200       1.000000   1.000000   1.000000   1.000000   1.000000